In [1]:
import warnings
warnings.filterwarnings('ignore')
import csv
import pandas as pd
import numpy as np
import random

from numpy import array
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import KFold
from sklearn import metrics

from sklearn import svm
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [2]:
title = ['Unigram+All','Logistic Regression','Random Forest','KNN','SVM','Decision Tree','Naive Bayes']
row1 = ['Accuracy']
row2 = ['Precision']
row3 = ['Recall']
row4 = ['Micro']
row5 = ['Macro']
row6 = ['Weighted']
row7 = ['AUC']

In [3]:
unigram = pd.read_csv('Pseudo3/Model_all/Unigram_MS.csv',encoding="UTF-8") 
data = pd.read_excel('Pseudo3/Model_all/Data_MS.xlsx',encoding="UTF-8") 
answer = pd.read_csv('Pseudo3/Model_all/Answer_MS.csv',encoding="UTF-8")
ntf = pd.read_excel('Pseudo3/NormalizedTF.xlsx',encoding="UTF-8")

In [4]:
result = pd.merge(unigram, data, on='Doc')

In [5]:
x_result = result.values
p_ntf = ntf.values
y_answer = answer.values

x = x_result[:,1:]
ps = p_ntf[:]
y = y_answer[:].ravel() 

b_Quantity = 50
nb_Quantity = 100
# scoring = ['accuracy', 'precision', 'recall', 'f1_micro', 'f1_macro', 'f1_weighted', 'roc_auc'] 

In [6]:
#LogisticRegression
acc = []
f1 = []
f2 = []
f3 = []
roc_auc = []
p =[]
r = []

kf = KFold(n_splits = 10)
for train_index, test_index in kf.split(x):
   #     print(" %s %s " % (train_index, test_index))
    doc = []
    ans = []
    rnd =[]
    x_cut = []
    y_cut = []
    #x_test
    x_test = np.array(x)[test_index]
    #y_test
    y_test = np.array(y)[test_index]
    #P_train
    p_train = np.array(ps)[train_index]
    #PB_train，檔名/答案
    p_B = sorted(p_train, key=lambda x:x[1], reverse=True)[:b_Quantity] #大到小排序`,取前B_Quantity個
    for b_n in range(len(p_B)):
        doc.append(p_B[b_n][0])
        ans.append([p_B[b_n][0],1])
    #PNB_train，檔名/答案
    p_NB = sorted(p_train, key=lambda x:x[1])
    for nb_n in range(len(p_NB)):
        if(p_NB[nb_n][1] == 0.0):
            rnd.append(p_NB[nb_n][0])
    random.shuffle(rnd)
    for rnd_n in range(0,nb_Quantity):
        doc.append(rnd[rnd_n])
        ans.append([rnd[rnd_n],0])
    #x_train
    for x_re in range(len(x_result)):
        if(x_result[x_re][0] in doc):
            x_cut.append(x_result[x_re])
    x_t = array(x_cut)
    x_train = x_t[:,1:]
    #y_train
    for y_re in range(len(x_t)):
        for y_as in range(len(ans)):
            if(x_t[y_re][0] == ans[y_as][0]):
                y_cut.append(ans[y_as][1])
    y_train = array(y_cut)
    
    lr = LogisticRegression()
    lr.fit(x_train,y_train)
    
    y_pred = np.array(lr.predict(x_test))
    y_score = np.array(lr.predict_proba(x_test)[:,1])
    y_test = np.array(y_test) #y_true
    acc.append(lr.score(x_test,y_test))
    f1.append(f1_score(y_test, y_pred, average='weighted'))
    f2.append(f1_score(y_test, y_pred, average='micro'))
    f3.append(f1_score(y_test, y_pred, average='macro'))
    roc_auc.append(roc_auc_score(y_test, y_score))
    p.append(precision_score(y_test, y_pred))
    r.append(recall_score(y_test, y_pred))
accuracy=0
weighted=0
micro=0
macro=0
auc=0
precision = 0
recall = 0

for i in range(0,10):
    accuracy=accuracy+acc[i]
    weighted=weighted+f1[i]
    micro=micro+f2[i]
    macro=macro+f3[i]
    precision=precision+p[i]
    recall=recall+r[i]
    auc=auc+roc_auc[i]
    
accuracy=accuracy/10
weighted=weighted/10
micro=micro/10
macro=macro/10
auc=auc/10
precision=precision/10
recall=recall/10

#accuracy
print("{:.12f}".format(accuracy))
row1.append(("{:.12f}".format(accuracy)))
#precision
print("{:.12f}".format(precision))
row2.append(("{:.12f}".format(precision)))
#recall
print("{:.12f}".format(recall))
row3.append(("{:.12f}".format(recall)))
#f-measure
print("{:.12f}".format(micro))
row4.append(("{:.12f}".format(micro)))
print("{:.12f}".format(macro))
row5.append(("{:.12f}".format(macro)))
print("{:.12f}".format(weighted))
row6.append(("{:.12f}".format(weighted)))
#auc
#fpr, tpr, thresholds = metrics.roc_curve(y,pred,pos_label=2)
#metrics.auc(fpr, tpr)
print("{:.12f}".format(auc))
row7.append(("{:.12f}".format(auc)))

0.781039360094
0.460279796053
0.290221328098
0.781039360094
0.610125895203
0.759526022449
0.637312503133


In [7]:
#RandomForestClassifier
acc = []
f1 = []
f2 = []
f3 = []
roc_auc = []
p =[]
r = []

kf = KFold(n_splits = 10)
for train_index, test_index in kf.split(x):
   #     print(" %s %s " % (train_index, test_index))
    doc = []
    ans = []
    rnd =[]
    x_cut = []
    y_cut = []
    #x_test
    x_test = np.array(x)[test_index]
    #y_test
    y_test = np.array(y)[test_index]
    #P_train
    p_train = np.array(ps)[train_index]
    #PB_train，檔名/答案
    p_B = sorted(p_train, key=lambda x:x[1], reverse=True)[:b_Quantity] #大到小排序`,取前B_Quantity個
    for b_n in range(len(p_B)):
        doc.append(p_B[b_n][0])
        ans.append([p_B[b_n][0],1])
    #PNB_train，檔名/答案
    p_NB = sorted(p_train, key=lambda x:x[1])
    for nb_n in range(len(p_NB)):
        if(p_NB[nb_n][1] == 0.0):
            rnd.append(p_NB[nb_n][0])
    random.shuffle(rnd)
    for rnd_n in range(0,nb_Quantity):
        doc.append(rnd[rnd_n])
        ans.append([rnd[rnd_n],0])
    #x_train
    for x_re in range(len(x_result)):
        if(x_result[x_re][0] in doc):
            x_cut.append(x_result[x_re])
    x_t = array(x_cut)
    x_train = x_t[:,1:]
    #y_train
    for y_re in range(len(x_t)):
        for y_as in range(len(ans)):
            if(x_t[y_re][0] == ans[y_as][0]):
                y_cut.append(ans[y_as][1])
    y_train = array(y_cut)
    
    rfc = RandomForestClassifier()
    rfc.fit(x_train,y_train)
    
    y_pred = np.array(rfc.predict(x_test)) #y_score
    y_score = np.array(rfc.predict_proba(x_test)[:,1])
    y_test = np.array(y_test) #y_true
    
    acc.append(rfc.score(x_test,y_test))
    f1.append(f1_score(y_test, y_pred, average='weighted'))
    f2.append(f1_score(y_test, y_pred, average='micro'))
    f3.append(f1_score(y_test, y_pred, average='macro'))
    roc_auc.append(roc_auc_score(y_test, y_score))
    p.append(precision_score(y_test, y_pred))
    r.append(recall_score(y_test, y_pred))
accuracy=0
weighted=0
micro=0
macro=0
auc=0
precision = 0
recall = 0

for i in range(0,10):
    accuracy=accuracy+acc[i]
    weighted=weighted+f1[i]
    micro=micro+f2[i]
    macro=macro+f3[i]
    precision=precision+p[i]
    recall=recall+r[i]
    auc=auc+roc_auc[i]   

accuracy=accuracy/10
weighted=weighted/10
micro=micro/10
macro=macro/10
auc=auc/10
precision=precision/10
recall=recall/10

#accuracy
print("{:.12f}".format(accuracy))
row1.append(("{:.12f}".format(accuracy)))
#precision
print("{:.12f}".format(precision))
row2.append(("{:.12f}".format(precision)))
#recall
print("{:.12f}".format(recall))
row3.append(("{:.12f}".format(recall)))
#f-measure
print("{:.12f}".format(micro))
row4.append(("{:.12f}".format(micro)))
print("{:.12f}".format(macro))
row5.append(("{:.12f}".format(macro)))
print("{:.12f}".format(weighted))
row6.append(("{:.12f}".format(weighted)))
#auc
#fpr, tpr, thresholds = metrics.roc_curve(y,pred,pos_label=2)
#metrics.auc(fpr, tpr)
print("{:.12f}".format(auc))
row7.append(("{:.12f}".format(auc)))

0.781046676096
0.435624186615
0.157973537396
0.781046676096
0.548035500478
0.735904674357
0.625485354777


In [8]:
#KNeighborsClassifier
acc = []
f1 = []
f2 = []
f3 = []
roc_auc = []
p =[]
r = []

kf = KFold(n_splits = 10)
for train_index, test_index in kf.split(x):
   #     print(" %s %s " % (train_index, test_index))
    doc = []
    ans = []
    rnd =[]
    x_cut = []
    y_cut = []
    #x_test
    x_test = np.array(x)[test_index]
    #y_test
    y_test = np.array(y)[test_index]
    #P_train
    p_train = np.array(ps)[train_index]
    #PB_train，檔名/答案
    p_B = sorted(p_train, key=lambda x:x[1], reverse=True)[:b_Quantity] #大到小排序`,取前B_Quantity個
    for b_n in range(len(p_B)):
        doc.append(p_B[b_n][0])
        ans.append([p_B[b_n][0],1])
    #PNB_train，檔名/答案
    p_NB = sorted(p_train, key=lambda x:x[1])
    for nb_n in range(len(p_NB)):
        if(p_NB[nb_n][1] == 0.0):
            rnd.append(p_NB[nb_n][0])
    random.shuffle(rnd)
    for rnd_n in range(0,nb_Quantity):
        doc.append(rnd[rnd_n])
        ans.append([rnd[rnd_n],0])
    #x_train
    for x_re in range(len(x_result)):
        if(x_result[x_re][0] in doc):
            x_cut.append(x_result[x_re])
    x_t = array(x_cut)
    x_train = x_t[:,1:]
    #y_train
    for y_re in range(len(x_t)):
        for y_as in range(len(ans)):
            if(x_t[y_re][0] == ans[y_as][0]):
                y_cut.append(ans[y_as][1])
    y_train = array(y_cut)
    
    knn = KNeighborsClassifier()
    knn.fit(x_train,y_train)
    
    y_pred = np.array(knn.predict(x_test)) #y_score
    y_score = np.array(knn.predict_proba(x_test)[:,1])
    y_test = np.array(y_test) #y_true
    
    acc.append(knn.score(x_test,y_test))
    f1.append(f1_score(y_test, y_pred, average='weighted'))
    f2.append(f1_score(y_test, y_pred, average='micro'))
    f3.append(f1_score(y_test, y_pred, average='macro'))
    roc_auc.append(roc_auc_score(y_test, y_score))
    p.append(precision_score(y_test, y_pred))
    r.append(recall_score(y_test, y_pred))
accuracy=0
weighted=0
micro=0
macro=0
auc=0
precision = 0
recall = 0

for i in range(0,10):
    accuracy=accuracy+acc[i]
    weighted=weighted+f1[i]
    micro=micro+f2[i]
    macro=macro+f3[i]
    precision=precision+p[i]
    recall=recall+r[i]
    auc=auc+roc_auc[i]
    
accuracy=accuracy/10
weighted=weighted/10
micro=micro/10
macro=macro/10
auc=auc/10
precision=precision/10
recall=recall/10

#accuracy
print("{:.12f}".format(accuracy))
row1.append(("{:.12f}".format(accuracy)))
#precision
print("{:.12f}".format(precision))
row2.append(("{:.12f}".format(precision)))
#recall
print("{:.12f}".format(recall))
row3.append(("{:.12f}".format(recall)))
#f-measure
print("{:.12f}".format(micro))
row4.append(("{:.12f}".format(micro)))
print("{:.12f}".format(macro))
row5.append(("{:.12f}".format(macro)))
print("{:.12f}".format(weighted))
row6.append(("{:.12f}".format(weighted)))
#auc
#fpr, tpr, thresholds = metrics.roc_curve(y,pred,pos_label=2)
#metrics.auc(fpr, tpr)
print("{:.12f}".format(auc))
row7.append(("{:.12f}".format(auc)))

0.705606496610
0.270585308828
0.226521260647
0.705606496610
0.530350336360
0.696590988884
0.523169491292


In [9]:
#SVM
acc = []
f1 = []
f2 = []
f3 = []
roc_auc = []
p =[]
r = []

kf = KFold(n_splits = 10)
for train_index, test_index in kf.split(x):
   #     print(" %s %s " % (train_index, test_index))
    doc = []
    ans = []
    rnd =[]
    x_cut = []
    y_cut = []
    #x_test
    x_test = np.array(x)[test_index]
    #y_test
    y_test = np.array(y)[test_index]
    #P_train
    p_train = np.array(ps)[train_index]
    #PB_train，檔名/答案
    p_B = sorted(p_train, key=lambda x:x[1], reverse=True)[:b_Quantity] #大到小排序`,取前B_Quantity個
    for b_n in range(len(p_B)):
        doc.append(p_B[b_n][0])
        ans.append([p_B[b_n][0],1])
    #PNB_train，檔名/答案
    p_NB = sorted(p_train, key=lambda x:x[1])
    for nb_n in range(len(p_NB)):
        if(p_NB[nb_n][1] == 0.0):
            rnd.append(p_NB[nb_n][0])
    random.shuffle(rnd)
    for rnd_n in range(0,nb_Quantity):
        doc.append(rnd[rnd_n])
        ans.append([rnd[rnd_n],0])
    #x_train
    for x_re in range(len(x_result)):
        if(x_result[x_re][0] in doc):
            x_cut.append(x_result[x_re])
    x_t = array(x_cut)
    x_train = x_t[:,1:]
    #y_train
    for y_re in range(len(x_t)):
        for y_as in range(len(ans)):
            if(x_t[y_re][0] == ans[y_as][0]):
                y_cut.append(ans[y_as][1])
    y_train = array(y_cut)
    
    sv = svm.SVC(kernel='linear')
    sv.fit(x_train,y_train)
    
    y_pred = np.array(sv.predict(x_test)) #y_score
    y_score = np.array(sv.decision_function(x_test))
    y_test = np.array(y_test) #y_true
    
    acc.append(sv.score(x_test,y_test))
    f1.append(f1_score(y_test, y_pred, average='weighted'))
    f2.append(f1_score(y_test, y_pred, average='micro'))
    f3.append(f1_score(y_test, y_pred, average='macro'))
    roc_auc.append(roc_auc_score(y_test, y_score))
    p.append(precision_score(y_test, y_pred))
    r.append(recall_score(y_test, y_pred))
accuracy=0
weighted=0
micro=0
macro=0
auc=0
precision = 0
recall = 0

for i in range(0,10):
    accuracy=accuracy+acc[i]
    weighted=weighted+f1[i]
    micro=micro+f2[i]
    macro=macro+f3[i]
    precision=precision+p[i]
    recall=recall+r[i]
    auc=auc+roc_auc[i]
    
accuracy=accuracy/10
weighted=weighted/10
micro=micro/10
macro=macro/10
auc=auc/10
precision=precision/10
recall=recall/10

#accuracy
print("{:.12f}".format(accuracy))
row1.append(("{:.12f}".format(accuracy)))
#precision
print("{:.12f}".format(precision))
row2.append(("{:.12f}".format(precision)))
#recall
print("{:.12f}".format(recall))
row3.append(("{:.12f}".format(recall)))
#f-measure
print("{:.12f}".format(micro))
row4.append(("{:.12f}".format(micro)))
print("{:.12f}".format(macro))
row5.append(("{:.12f}".format(macro)))
print("{:.12f}".format(weighted))
row6.append(("{:.12f}".format(weighted)))
#auc
#fpr, tpr, thresholds = metrics.roc_curve(y,pred,pos_label=2)
#metrics.auc(fpr, tpr)
print("{:.12f}".format(auc))
row7.append(("{:.12f}".format(auc)))

0.754423742867
0.407476046059
0.376876456849
0.754423742867
0.617240406950
0.749712685797
0.656625429037


In [10]:
#DecisionTreeClassifier
acc = []
f1 = []
f2 = []
f3 = []
roc_auc = []
p =[]
r = []

kf = KFold(n_splits = 10)
for train_index, test_index in kf.split(x):
   #     print(" %s %s " % (train_index, test_index))
    doc = []
    ans = []
    rnd =[]
    x_cut = []
    y_cut = []
    #x_test
    x_test = np.array(x)[test_index]
    #y_test
    y_test = np.array(y)[test_index]
    #P_train
    p_train = np.array(ps)[train_index]
    #PB_train，檔名/答案
    p_B = sorted(p_train, key=lambda x:x[1], reverse=True)[:b_Quantity] #大到小排序`,取前B_Quantity個
    for b_n in range(len(p_B)):
        doc.append(p_B[b_n][0])
        ans.append([p_B[b_n][0],1])
    #PNB_train，檔名/答案
    p_NB = sorted(p_train, key=lambda x:x[1])
    for nb_n in range(len(p_NB)):
        if(p_NB[nb_n][1] == 0.0):
            rnd.append(p_NB[nb_n][0])
    random.shuffle(rnd)
    for rnd_n in range(0,nb_Quantity):
        doc.append(rnd[rnd_n])
        ans.append([rnd[rnd_n],0])
    #x_train
    for x_re in range(len(x_result)):
        if(x_result[x_re][0] in doc):
            x_cut.append(x_result[x_re])
    x_t = array(x_cut)
    x_train = x_t[:,1:]
    #y_train
    for y_re in range(len(x_t)):
        for y_as in range(len(ans)):
            if(x_t[y_re][0] == ans[y_as][0]):
                y_cut.append(ans[y_as][1])
    y_train = array(y_cut)
    
    dtc = DecisionTreeClassifier() 
    dtc.fit(x_train,y_train)
    
    y_pred = np.array(dtc.predict(x_test)) #y_score
    y_score = np.array(dtc.predict_proba(x_test)[:,1])
    y_test = np.array(y_test) #y_true
    
    acc.append(dtc.score(x_test,y_test))
    f1.append(f1_score(y_test, y_pred, average='weighted'))
    f2.append(f1_score(y_test, y_pred, average='micro'))
    f3.append(f1_score(y_test, y_pred, average='macro'))
    roc_auc.append(roc_auc_score(y_test, y_score))
    p.append(precision_score(y_test, y_pred))
    r.append(recall_score(y_test, y_pred))
accuracy=0
weighted=0
micro=0
macro=0
auc=0
precision = 0
recall = 0

for i in range(0,10):
    accuracy=accuracy+acc[i]
    weighted=weighted+f1[i]
    micro=micro+f2[i]
    macro=macro+f3[i]
    precision=precision+p[i]
    recall=recall+r[i]
    auc=auc+roc_auc[i]
    
accuracy=accuracy/10
weighted=weighted/10
micro=micro/10
macro=macro/10
auc=auc/10
precision=precision/10
recall=recall/10

#accuracy
print("{:.12f}".format(accuracy))
row1.append(("{:.12f}".format(accuracy)))
#precision
print("{:.12f}".format(precision))
row2.append(("{:.12f}".format(precision)))
#recall
print("{:.12f}".format(recall))
row3.append(("{:.12f}".format(recall)))
#f-measure
print("{:.12f}".format(micro))
row4.append(("{:.12f}".format(micro)))
print("{:.12f}".format(macro))
row5.append(("{:.12f}".format(macro)))
print("{:.12f}".format(weighted))
row6.append(("{:.12f}".format(weighted)))
#auc
#fpr, tpr, thresholds = metrics.roc_curve(y,pred,pos_label=2)
#metrics.auc(fpr, tpr)
print("{:.12f}".format(auc))
row7.append(("{:.12f}".format(auc)))

0.680966200068
0.341328645657
0.559376109959
0.680966200068
0.601046082454
0.704332509117
0.636561951357


In [11]:
#GaussianNB
acc = []
f1 = []
f2 = []
f3 = []
roc_auc = []
p =[]
r = []

kf = KFold(n_splits = 10)
for train_index, test_index in kf.split(x):
   #     print(" %s %s " % (train_index, test_index))
    doc = []
    ans = []
    rnd =[]
    x_cut = []
    y_cut = []
    #x_test
    x_test = np.array(x)[test_index]
    #y_test
    y_test = np.array(y)[test_index]
    #P_train
    p_train = np.array(ps)[train_index]
    #PB_train，檔名/答案
    p_B = sorted(p_train, key=lambda x:x[1], reverse=True)[:b_Quantity] #大到小排序`,取前B_Quantity個
    for b_n in range(len(p_B)):
        doc.append(p_B[b_n][0])
        ans.append([p_B[b_n][0],1])
    #PNB_train，檔名/答案
    p_NB = sorted(p_train, key=lambda x:x[1])
    for nb_n in range(len(p_NB)):
        if(p_NB[nb_n][1] == 0.0):
            rnd.append(p_NB[nb_n][0])
    random.shuffle(rnd)
    for rnd_n in range(0,nb_Quantity):
        doc.append(rnd[rnd_n])
        ans.append([rnd[rnd_n],0])
    #x_train
    for x_re in range(len(x_result)):
        if(x_result[x_re][0] in doc):
            x_cut.append(x_result[x_re])
    x_t = array(x_cut)
    x_train = x_t[:,1:]
    #y_train
    for y_re in range(len(x_t)):
        for y_as in range(len(ans)):
            if(x_t[y_re][0] == ans[y_as][0]):
                y_cut.append(ans[y_as][1])
    y_train = array(y_cut)
    
    gnb = GaussianNB() 
    gnb.fit(x_train,y_train)
    
    y_pred = np.array(gnb.predict(x_test)) #y_score
    y_score = np.array(gnb.predict_proba(x_test)[:,1])
    y_test = np.array(y_test) #y_true
    
    acc.append(gnb.score(x_test,y_test))
    f1.append(f1_score(y_test, y_pred, average='weighted'))
    f2.append(f1_score(y_test, y_pred, average='micro'))
    f3.append(f1_score(y_test, y_pred, average='macro'))
    roc_auc.append(roc_auc_score(y_test, y_score))
    p.append(precision_score(y_test, y_pred))
    r.append(recall_score(y_test, y_pred))
accuracy=0
weighted=0
micro=0
macro=0
auc=0
precision = 0
recall = 0

for i in range(0,10):
    accuracy=accuracy+acc[i]
    weighted=weighted+f1[i]
    micro=micro+f2[i]
    macro=macro+f3[i]
    precision=precision+p[i]
    recall=recall+r[i]
    auc=auc+roc_auc[i]
    
accuracy=accuracy/10
weighted=weighted/10
micro=micro/10
macro=macro/10
auc=auc/10
precision=precision/10
recall=recall/10

#accuracy
print("{:.12f}".format(accuracy))
row1.append(("{:.12f}".format(accuracy)))
#precision
print("{:.12f}".format(precision))
row2.append(("{:.12f}".format(precision)))
#recall
print("{:.12f}".format(recall))
row3.append(("{:.12f}".format(recall)))
#f-measure
print("{:.12f}".format(micro))
row4.append(("{:.12f}".format(micro)))
print("{:.12f}".format(macro))
row5.append(("{:.12f}".format(macro)))
print("{:.12f}".format(weighted))
row6.append(("{:.12f}".format(weighted)))
#auc
#fpr, tpr, thresholds = metrics.roc_curve(y,pred,pos_label=2)
#metrics.auc(fpr, tpr)
print("{:.12f}".format(auc))
row7.append(("{:.12f}".format(auc)))

0.696217626689
0.337709969873
0.458067208742
0.696217626689
0.591977843785
0.711362405992
0.609274938372


In [12]:
with open('Pseudo3/NTF/NTF510.csv', 'a', encoding='UTF-8', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(title)
    writer.writerow(row1)
    writer.writerow(row2)
    writer.writerow(row3)
    writer.writerow(row4)
    writer.writerow(row5)
    writer.writerow(row6)
    writer.writerow(row7)